Step 1: Data Cleaning and Saving Cleaned Versions

In [9]:
import pandas as pd
import numpy as np

creditcard_df = pd.read_csv('../data/creditcard.csv')
fraud_data_df = pd.read_csv('../data/Fraud_Data.csv')

In [10]:
creditcard_df.drop_duplicates(inplace=True)

In [11]:
creditcard_df['Time'] = pd.to_datetime(creditcard_df['Time'], unit='s', origin='2010-01-01')

In [12]:
creditcard_df.to_csv('../data/cleaned_creditcard.csv', index=False)

In [13]:
fraud_data_df.drop_duplicates(inplace=True)

In [14]:
fraud_data_df['signup_time'] = pd.to_datetime(fraud_data_df['signup_time'])
fraud_data_df['purchase_time'] = pd.to_datetime(fraud_data_df['purchase_time'])

In [15]:
fraud_data_df.to_csv('../data/cleaned_fraud_data.csv', index=False)

Feature and Target Separation

In [16]:
X_credit = creditcard_df.drop(columns=['Class'])
y_credit = creditcard_df['Class']

X_fraud = fraud_data_df.drop(columns=['class'])
y_fraud = fraud_data_df['class']

Train-Test Split

In [17]:
from sklearn.model_selection import train_test_split

X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(
    X_credit, y_credit, test_size=0.2, random_state=42, stratify=y_credit)

X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud)